In [1]:
#To remove all variables from the namespace
%reset -f

#Creating a log file to record the commands and outputs
%logstop
%logstart -t -o "E:/Python Clinical Course/ADLB log.txt"

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : E:/Python Clinical Course/ADLB log.txt
Mode           : backup
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [2]:
import pandas as pd
import numpy as np
import os
import pyreadstat
from datetime import datetime
from pathlib import Path

# Define paths (adjust these to your actual paths)
adam_path = r"E:\Python Clinical Course\ADAM datasets\ADaM Datasets"
sdtm_path = r"E:\Python Clinical Course\SDTM"
raw_path = r"E:\Python Clinical Course\RAW"


In [3]:
sdtm_datasets = {}

for file in os.listdir(sdtm_path):
    if file.endswith(".sas7bdat"):
        dataset_name = file.replace(".sas7bdat", "")
        file_path = os.path.join(sdtm_path, file)
        df, meta = pyreadstat.read_sas7bdat(file_path)
        sdtm_datasets[dataset_name] = df

In [4]:

# 1) Read the main SDTM dataset
lb1 = sdtm_datasets.get("lb")
lb1 = lb1.drop(columns=['ARM', 'ACTARM'], errors='ignore')
lb1 = lb1.sort_values(['USUBJID', 'LBSEQ']).reset_index(drop=True)


In [5]:

# 2) Process SUPPLB dataset
supplb = sdtm_datasets.get("supplb")
supplb['LBSEQ'] = pd.to_numeric(supplb['IDVARVAL'], errors='coerce')
supplb = supplb.sort_values(['USUBJID', 'LBSEQ']).reset_index(drop=True)


In [6]:

# Transpose SUPPLB
supplb_pivot = supplb.pivot_table(
    index=['USUBJID', 'LBSEQ'], 
    columns='QNAM', 
    values='QVAL', 
    aggfunc='first'
).reset_index()


In [7]:

# 3) Merge LB with transposed SUPPLB
lb2 = pd.merge(lb1, supplb_pivot, on=['USUBJID', 'LBSEQ'], how='left')


In [9]:
adam_datasets = {}

for file in os.listdir(adam_path):
    if file.endswith(".sas7bdat"):
        dataset_name = file.replace(".sas7bdat", "")
        file_path = os.path.join(adam_path, file)
        df, meta = pyreadstat.read_sas7bdat(file_path)
        adam_datasets[dataset_name] = df


In [10]:

# 4) Merge with ADSL
adsl = adam_datasets.get("adsl")
lb3 = pd.merge(lb2, adsl, on='USUBJID', how='inner')


In [11]:

# 5) Create derived variables
lb4 = lb3.copy()

# PARCAT1
lb4['PARCAT1'] = lb4['LBCAT']

# AVISIT and AVISITN logic
lb4['AVISIT'] = lb4['VISIT']
lb4['AVISITN'] = lb4['VISITNUM']


In [12]:

# Handle unscheduled visits
unscheduled_mask = lb4['VISIT'].str.contains('UNSCHEDULED', case=False, na=False)
lb4.loc[unscheduled_mask, 'AVISIT'] = np.nan
lb4.loc[unscheduled_mask, 'AVISITN'] = np.nan


In [13]:

# Baseline flag logic
baseline_mask = lb4['LBBLFL'] == 'Y'
lb4.loc[baseline_mask, 'AVISIT'] = 'Baseline'
lb4.loc[baseline_mask, 'AVISITN'] = 1


In [14]:

# PARAM creation
def create_param(row):
    if pd.notna(row['LBSTRESU']) and row['LBSTRESU'] != '':
        return f"{row['LBTEST'].strip()} ({row['LBSTRESU'].strip()})"
    else:
        return row['LBTEST'].strip() if pd.notna(row['LBTEST']) else ''

lb4['PARAM'] = lb4.apply(create_param, axis=1)
lb4['PARAMCD'] = lb4['LBTESTCD']

# Standard range variables
lb4['ANRLO'] = lb4['LBSTNRLO']
lb4['ANRHI'] = lb4['LBSTNRHI']
lb4['ANRIND'] = lb4['LBNRIND']

# Treatment variables
lb4['TRTP'] = lb4['TRT01P']
lb4['TRTPN'] = lb4['TRT01PN']
lb4['TRTA'] = lb4['TRT01A']
lb4['TRTAN'] = lb4['TRT01AN']

# Analysis values
lb4['AVAL'] = lb4['LBSTRESN']
lb4['AVALC'] = lb4['LBSTRESC']


In [16]:

def process_datetime(dtc_str):
    if pd.isna(dtc_str) or dtc_str == '':
        return None, None, None

    try:
        dt = pd.to_datetime(dtc_str, errors='coerce')
        if pd.isna(dt):
            return None, None, None
        date_part = dt.date()
        time_part = dt.time() if 'T' in str(dtc_str) else None
        return date_part, dt, time_part
    except Exception:
        return None, None, None

# Apply datetime processing
datetime_results = lb4['LBDTC'].apply(process_datetime)
lb4['ADT'] = [x[0] for x in datetime_results]
lb4['ADTM'] = [x[1] for x in datetime_results]
lb4['ATM'] = [x[2] for x in datetime_results]


In [17]:

# Study day calculation
def calculate_ady(adt, trtsdt):
    if pd.isna(adt) or pd.isna(trtsdt):
        return np.nan
    
    adt = pd.to_datetime(adt)
    trtsdt = pd.to_datetime(trtsdt)
    
    if adt < trtsdt:
        return (adt - trtsdt).days
    else:
        return (adt - trtsdt).days + 1

lb4['ADY'] = lb4.apply(lambda row: calculate_ady(row['ADT'], row['TRTSDT']), axis=1)


In [18]:

# 6) Create PARAMN

lb4 = lb4.sort_values('LBTESTCD').reset_index(drop=True)
param_map = pd.DataFrame({'LBTESTCD': lb4['LBTESTCD'].unique()})
param_map['PARAMN'] = range(1, len(param_map) + 1)
lb4 = pd.merge(lb4, param_map, on='LBTESTCD', how='left')


In [19]:

# 7) Baseline processing
lb4 = lb4.sort_values(['USUBJID', 'PARAMN', 'AVISITN', 'ADT']).reset_index(drop=True)

# Create subset for baseline calculation
lb5 = lb4[['USUBJID', 'PARAMN', 'PARAM', 'ADT', 'ADTM', 'AVAL', 'AVALC', 'TRTSDT', 'AVISITN']].copy()


In [20]:

# Filter for baseline candidates (before or on treatment start date)
lb6 = lb5[
    (lb5['ADT'].notna()) & 
    (lb5['ADT'] <= lb5['TRTSDT']) & 
    ((lb5['AVAL'].notna()) | (lb5['AVALC'].notna() & (lb5['AVALC'] != '')))
].copy()

# Get last observation per parameter before treatment
lb7 = lb6.groupby(['USUBJID', 'PARAMN']).last().reset_index()
lb7 = lb7[['USUBJID', 'PARAMN', 'AVISITN']]

# Mark baseline records
lb8 = pd.merge(lb4, lb7, on=['USUBJID', 'PARAMN', 'AVISITN'], how='left', indicator=True)
lb8['ABLFL'] = np.where(lb8['_merge'] == 'both', 'Y', '')
lb8 = lb8.drop('_merge', axis=1)


In [21]:

# 8) Calculate baseline values and changes
lb9 = lb8.sort_values(['USUBJID', 'PARAMN', 'AVISITN']).reset_index(drop=True)

# Initialize baseline columns
lb9['BASE'] = np.nan
lb9['BASEC'] = ''
lb9['BNRIND'] = ''
lb9['CHG'] = np.nan
lb9['PCHG'] = np.nan

# Calculate baseline values and changes
for (usubjid, paramn), group in lb9.groupby(['USUBJID', 'PARAMN']):
    baseline_idx = group[group['ABLFL'] == 'Y'].index
    
    if len(baseline_idx) > 0:
        baseline_row = group.loc[baseline_idx[0]]
        base_val = baseline_row['AVAL']
        base_char = baseline_row['AVALC']
        base_nrind = baseline_row['LBNRIND']
        
        # Set baseline values for all records of this parameter
        lb9.loc[group.index, 'BASE'] = base_val
        lb9.loc[group.index, 'BASEC'] = base_char
        lb9.loc[group.index, 'BNRIND'] = base_nrind
        
        # Calculate changes for non-baseline records
        non_baseline_idx = group[group['ABLFL'] != 'Y'].index
        for idx in non_baseline_idx:
            aval = lb9.loc[idx, 'AVAL']
            if pd.notna(aval) and pd.notna(base_val):
                lb9.loc[idx, 'CHG'] = aval - base_val
                if base_val != 0:
                    lb9.loc[idx, 'PCHG'] = ((aval - base_val) / base_val) * 100

# ANL01FL flag for scheduled visits
lb9['ANL01FL'] = np.where(
    ~lb9['VISIT'].str.contains('UNSCHEDULED', case=False, na=False), 
    'Y', 
    ''
)


In [22]:

# 9) Final dataset with proper column order
final_columns = [
    'STUDYID', 'USUBJID', 'SUBJID', 'SITEID', 'AGE', 'AGEU', 'SEX', 'RACE', 
    'ETHNIC', 'COUNTRY', 'SAFFL', 'ITTFL', 'PPROTFL', 'RANDFL', 'TRTP', 'TRTPN', 
    'TRTA', 'TRTAN', 'TRTSDTM', 'TRTSDT', 'TRTEDTM', 'TRTEDT', 'ADTM', 'ADT', 
    'ATM', 'ADY', 'PARCAT1', 'PARAM', 'PARAMN', 'PARAMCD', 'AVAL', 'AVALC', 
    'ABLFL', 'BASE', 'BASEC', 'CHG', 'PCHG', 'ANRLO', 'ANRHI', 'ANRIND', 
    'BNRIND', 'VISITNUM', 'VISIT', 'AVISIT', 'AVISITN', 'ANL01FL'
]


In [23]:
existing_columns = [col for col in final_columns if col in lb9.columns]

In [24]:
lb10 = lb9[existing_columns].copy()

In [25]:
column_labels = {
'STUDYID': "Study Identifier",
'USUBJID': "Unique Subject Identifier",
'SUBJID': "Subject Identifier for the Study",
'SITEID': "Study Site Identifier",
'AGE': "Age",
'AGEU': "Age Units",
'SEX': "Sex",
'RACE': "Race",
'ETHNIC': "Ethnicity",
'COUNTRY': "Country",
'SAFFL': "Safety Population Flag",
'ITTFL': "Intent-To-Treat Population Flag",
'PPROTFL': "Per-Protocol Population Flag",
'RANDFL': "Randomized Population Flag",
'TRTP': "Planned Treatment",
'TRTPN': "Planned Treatment (N)",
'TRTA': "Actual Treatment",
'TRTAN': "Actual Treatment (N)",
'TRTSDTM': "Datetime of First Exposure to Treatment",
'TRTSDT': "Date of First Exposure to Treatment",
'TRTEDTM': "Datetime of Last Exposure to Treatment",
'TRTEDT': "Date of Last Exposure to Treatment",
'ADTM ': "Analysis Datetime",
'ADT ': "Analysis Date",
'ATM ': "Analysis Time",
'ADY ': "Analysis Relative Day",
'PARCAT1 ': "Parameter Category 1",
'PARAM ': "Parameter",
'PARAMN': "Parameter (N)",
'PARAMCD ': "Parameter Code",
'AVAL': "Analysis Value",
'AVALC ': "Analysis Value (C)",
'ABLFL ': "Baseline Record Flag",
'BASE ': "Baseline Value",
'BASEC ': "Baseline Value (C)",
'CHG ': "Change from Baseline",
'PCHG ': "Percent Change from Baseline",
'ANRLO': "Analysis Normal Range Lower Limit",
'ANRHI': "Analysis Normal Range Upper Limit",
'ANRIND': "Analysis Reference Range Indicator",
'BNRIND': "Baseline Reference Range Indicator",
'VISITNUM ': "Visit Number",
'VISIT ': "Visit Name",
'AVISIT ': "Analysis Visit",
'AVISITN ': "Analysis Visit (N)",
'ANL01FL': "Analysis Flag 01",
}

In [26]:
lb10.attrs['column_labels'] = column_labels

In [27]:
for col in lb10.columns:
    label = lb10.attrs.get('column_labels', {}).get(col, '')
    print(f"{col}: {label}")

USUBJID: Unique Subject Identifier
SUBJID: Subject Identifier for the Study
SITEID: Study Site Identifier
AGE: Age
AGEU: Age Units
SEX: Sex
RACE: Race
ETHNIC: Ethnicity
COUNTRY: Country
SAFFL: Safety Population Flag
ITTFL: Intent-To-Treat Population Flag
PPROTFL: Per-Protocol Population Flag
RANDFL: Randomized Population Flag
TRTP: Planned Treatment
TRTPN: Planned Treatment (N)
TRTA: Actual Treatment
TRTAN: Actual Treatment (N)
TRTSDTM: Datetime of First Exposure to Treatment
TRTSDT: Date of First Exposure to Treatment
TRTEDTM: Datetime of Last Exposure to Treatment
TRTEDT: Date of Last Exposure to Treatment
ADTM: 
ADT: 
ATM: 
ADY: 
PARCAT1: 
PARAM: 
PARAMN: Parameter (N)
PARAMCD: 
AVAL: Analysis Value
AVALC: 
ABLFL: 
BASE: 
BASEC: 
CHG: 
PCHG: 
ANRLO: Analysis Normal Range Lower Limit
ANRHI: Analysis Normal Range Upper Limit
ANRIND: Analysis Reference Range Indicator
BNRIND: Baseline Reference Range Indicator
VISITNUM: 
VISIT: 
AVISIT: 
AVISITN: 
ANL01FL: Analysis Flag 01


In [28]:

#  Save the final dataset
output = "E:/Python Clinical Course/ADAM datasets"
output_path = f"{output}/ADLB.csv"
lb10.to_csv(output_path, index=False)

print(f"ADLB dataset created successfully with {len(lb10)} subjects and {len(lb10.columns)} variables.")
print(f"Dataset saved to: {output_path}")

# Display basic info about the dataset
print("\nDataset Info:")
print(f"Shape: {lb10.shape}")
print(f"Columns: {list(lb10.columns)}")


ADLB dataset created successfully with 2938 subjects and 45 variables.
Dataset saved to: E:/Python Clinical Course/ADAM datasets/ADLB.csv

Dataset Info:
Shape: (2938, 45)
Columns: ['USUBJID', 'SUBJID', 'SITEID', 'AGE', 'AGEU', 'SEX', 'RACE', 'ETHNIC', 'COUNTRY', 'SAFFL', 'ITTFL', 'PPROTFL', 'RANDFL', 'TRTP', 'TRTPN', 'TRTA', 'TRTAN', 'TRTSDTM', 'TRTSDT', 'TRTEDTM', 'TRTEDT', 'ADTM', 'ADT', 'ATM', 'ADY', 'PARCAT1', 'PARAM', 'PARAMN', 'PARAMCD', 'AVAL', 'AVALC', 'ABLFL', 'BASE', 'BASEC', 'CHG', 'PCHG', 'ANRLO', 'ANRHI', 'ANRIND', 'BNRIND', 'VISITNUM', 'VISIT', 'AVISIT', 'AVISITN', 'ANL01FL']
